# Organize variants selected from select_crispey3_library_variants notebook into pools
The goal is to assign each variant with a pool number, and that number will be used when assigning barcodes to guide-donors and grouping the oligos into pools in the final CRISPEY3 library construction.

There are multiple sets going into the final library, and each set will have slightly different grouping and randomization requirements.

Broadly speaking, here are some general principles:
- Keep oligos targeting the same variant together. This way, we control the pool covariate and can better account for guide-specific differences per variant.
- Within each variant set, avoid clustering variants in the same genomic region together. This avoids cloning issues (PCR mispriming, template swaps in Gibson assembly, etc.), and also controls for pool-specific batch effects when comparing fitness effects across variants.
    - This negates a gene-based grouping approach, which would make for convenient organization and effective utilization of pools to do gene-specific small growth competition experiments.
    - Sets such as the ergosterol pathway set and GxG sets must utilize whole-set variant randomization to avoid pool effects being associated with each gene.
- An alternative approach would be to pool variants by gene. The GxE set is already somewhat grouped by gene because although the variants are currently grouped by condition, genes are often condition-specific in their testing and so variants in the same gene will inevitably end up in the same few pools. Some randomization here might still be possible, but not if there are too few pools available, especially for oligos used in multiple conditions
- When assigning variants to each pool, take into account the number of oligos generated for each variant so that each pool contains the appropriate number of oligos (118, plus 3 additional technical oligos)

In [1]:
import os, random, vcf
import numpy as np
import pandas as pd

In [2]:
def sort_variants_to_pools(vars_df, assignment_order=[], grouping_col = None, num_of_pools_per_group={}, 
                           spillover_assignment={}, max_pool_size=1, first_pool=1, ):
    
    # for storing variant ID of spillover variants
    spillover_vars = {g : [] for g in assignment_order}
#     # add discard list
#     spillover_vars['discard'] = []
    
    # iterate over each group and assign to pools
    for group in assignment_order:
        # get subset of variants belonging to group
        # if column to group by provided, select variant subset by column. Otherwise, filter by variant ID
        if grouping_col:
            subset = vars_df.loc[vars_df[grouping_col] == group]
        else:
            subset = vars_df.filter(like=group, axis=0)
        # sort variants in descending order of number of oligos, then randomize variants within each group of a certain number of oligos
        # this enables even, efficient packing with randomization across pools
        subset = subset.groupby('num_of_oligos').apply(lambda x: x.sample(frac=1, random_state=1)).reset_index(level=0, drop=True)[::-1]
        
        # get number of pools to use for group
        num_of_pools = num_of_pools_per_group[group]
        # assign oligos to spillover if no pools set for group
        if num_of_pools == 0:
#             print("Excess oligos for {}: {}".format(group, subset.num_of_oligos.sum())) # debugging
            spillover_vars[spillover_assignment[group]] += subset.index.tolist()
            continue

        # define current pool sizes
        cur_pool_sizes = [0] * num_of_pools
        # add spillover variants to first pool(s) in group
        # note: packing is done by max-ing out first pool, then moving to next pool, and doing so until all spillover variants are assigned
        if len(spillover_vars[group]) > 0:
            spillover_pool_num = first_pool
            for v, row in vars_df.loc[spillover_vars[group]].iterrows():
                # move to next spillover pool number if pool has reached capacity
                if cur_pool_sizes[spillover_pool_num-first_pool] + row['num_of_oligos'] > max_pool_size:
                    spillover_pool_num += 1
                    
                # assign each spillover variant to pool number
                vars_df.loc[v, 'pool_num'] = spillover_pool_num
                # increase pool size by variant's number of oligos
                cur_pool_sizes[spillover_pool_num-first_pool] += row['num_of_oligos']
       
        # add all other variants to pools in group
        # variants are assigned to spread oligos out, ensuring similar group sizes across pools
        excess = 0
        for v, row in subset.iterrows():
            # find smallest current pool
            i = np.argmin(cur_pool_sizes)
            # assign variant to pool if number of oligos in pool is within max pool size
            if cur_pool_sizes[i] + row['num_of_oligos'] <= max_pool_size:
                vars_df.loc[v, 'pool_num'] = first_pool+i
                cur_pool_sizes[i] += row['num_of_oligos']
            # if current pools have insufficient space, send to group assigned for spillover
            else:
                excess +=row['num_of_oligos']
                spillover_vars[spillover_assignment[group]].append(v)
        
#         # (debugging) print spillover variants
#         if group in spillover_assignment.keys():
#             print("Excess oligos for {}: {}".format(group, excess))
        
        # once assigned, advance first_pool for next group
        first_pool += num_of_pools
    
#     # remove discarded variants
#     vars_df = vars_df.loc[~vars_df.index.isin(spillover_vars['discard'])]
#     # identify number of variants discarded
#     print("Variants discarded:", spillover_vars['discard'])

    vars_df['pool_num'] = vars_df['pool_num'].astype(int)

    return vars_df

In [3]:
# set working directory
working_dir = "/home/users/rang/crispey3/library_design/Input/"
os.chdir(working_dir)

# setting the random seed
random.seed(1)

## Ergosterol set
- ERG: 21 pools, 117-118 oligos each
- EGE: 4 pools, 117-118 oligos each
- EGD: 1 pool, 118 oligos each
- EGC: 2 pools, 110-111 oligos each
- EGB: 1 pool, 117 oligos each
- EGA: 1 pool, 117 oligos each
- EG9: 1 pool, 118 oligos each
- EG8: 3 pools, 113-114 oligos each

See code for spillover assignment

In [4]:
ergosterol_vars_file = "/home/users/rang/scratch/yeast/ergosterol/ergosterol_variants_final.txt"
ergosterol_vars = pd.read_csv(ergosterol_vars_file, sep='\t', index_col=0)
ergosterol_vars['num_of_oligos'] = ergosterol_vars['num_of_oligos'].astype(int)

# assign ergosterol variants to pools
ergosterol_vars = sort_variants_to_pools(vars_df = ergosterol_vars, 
                                         assignment_order = ['ERG','EGE','EGD','EGC','EGB','EGA','EG9','EG8'],
                                         num_of_pools_per_group = {'ERG':21, 'EGE':4, 'EGD':1, 'EGC':2, 'EGB':1, 'EGA':1, 'EG9':1, 'EG8':3},
                                         spillover_assignment = {'ERG':'EGE', 'EGE':'EGC', 'EGD':'EGC', 'EGC':'EG8', 'EGB':'EGA', 'EGA':'EG8', 'EG9':'EG8'},
                                         max_pool_size = 118, first_pool = 1)

display(ergosterol_vars)
# check number of oligos in each pool
display(ergosterol_vars.groupby('pool_num').num_of_oligos.sum())
print("Total number of oligos:", ergosterol_vars.num_of_oligos.sum())
print("Total number of variants:", len(ergosterol_vars))

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance,num_of_oligos,pool_num
var_id,,,,,,,,,,,,,,,,,,,,,
ERG_00003,V,237122,G,A,9,2022,0.004451,3,2382|1631|1338|448|2910,unidirectional_promoter,...,SAH1,YER043C,upstream_gene_variant,3.0,ERG28,YER044C,downstream_gene_variant,448.0,2,13
ERG_00004,V,237136,T,C,26,2022,0.013000,17,2396|1645|1324|434|2896,unidirectional_promoter,...,SAH1,YER043C,upstream_gene_variant,17.0,ERG28,YER044C,downstream_gene_variant,434.0,2,21
EGE_00008,V,237191,T,TG,661,2022,0.327000,72,2451|1700|1268|378|2840,unidirectional_promoter,...,SAH1,YER043C,upstream_gene_variant,72.0,ERG28,YER044C,downstream_gene_variant,378.0,2,25
EGE_00009,V,237191,T,TA,6,2022,0.002967,72,2451|1700|1268|378|2840,unidirectional_promoter,...,SAH1,YER043C,upstream_gene_variant,72.0,ERG28,YER044C,downstream_gene_variant,378.0,2,22
EGE_00010,V,237196,A,T,69,2022,0.034000,77,2456|1705|1264|374|2836,unidirectional_promoter,...,SAH1,YER043C,upstream_gene_variant,77.0,ERG28,YER044C,downstream_gene_variant,374.0,2,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EGA_00039,XVI,499649,G,A,4,2022,0.001978,3868|16,4690|1026|357|1930|4403,unidirectional_promoter,...,ERG10,YPL028W,downstream_gene_variant,357.0,SMA1,YPL027W,upstream_gene_variant,16.0,2,30
EGA_00040,XVI,499650,G,C,5,2022,0.002473,3867|15,4689|1025|358|1931|4404,unidirectional_promoter,...,ERG10,YPL028W,downstream_gene_variant,358.0,SMA1,YPL027W,upstream_gene_variant,15.0,2,30
EGA_00041,XVI,499655,G,A,4,2022,0.001978,3862|10,4684|1020|363|1936|4409,unidirectional_promoter,...,ERG10,YPL028W,downstream_gene_variant,363.0,SMA1,YPL027W,upstream_gene_variant,10.0,2,30


pool_num
1     118
2     118
3     118
4     118
5     118
6     118
7     118
8     118
9     117
10    117
11    117
12    117
13    117
14    117
15    117
16    117
17    117
18    117
19    117
20    117
21    117
22    118
23    118
24    118
25    117
26    118
27    110
28    111
29    117
30    117
31    118
32    114
33    114
34    113
Name: num_of_oligos, dtype: int64

Total number of oligos: 3968
Total number of variants: 1580


## GXG set
- GXG: 43 pools, 114-116 oligos each

In [5]:
gxg_vars_file = "/home/users/rang/scratch/yeast/genetic_interactions/costanzo_2016/gxg_variants_final.txt"
gxg_vars = pd.read_csv(gxg_vars_file, sep='\t', index_col=0)
gxg_vars['num_of_oligos'] = gxg_vars['num_of_oligos'].astype(int)

# assign gxg variants to pools
gxg_vars = sort_variants_to_pools(vars_df = gxg_vars, 
                                  assignment_order = ['GXG'],
                                  num_of_pools_per_group = {'GXG':43},
                                  spillover_assignment = {},
                                  max_pool_size = 118, first_pool = 35)

display(gxg_vars)
# check number of oligos in each pool
display(gxg_vars.groupby('pool_num').num_of_oligos.sum())
print("Total number of oligos:", gxg_vars.num_of_oligos.sum())
print("Total number of variants:", len(gxg_vars))

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance,num_of_oligos,assoc_gene,pool_num
var_id,,,,,,,,,,,,,,,,,,,,,
GXG_00321,I,101530,G,C,4,2022,0.001978,385|1833,1662|4742|35,unidirectional_promoter,...,YAL025C,upstream_gene_variant,385.0,LTE1,YAL024C,downstream_gene_variant,35.0,3,"['YAL024C', 'YAL025C']",68
GXG_00329,I,101816,C,T,4,2020,0.001980,671|2119,1948|4456,missense_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,['YAL024C'],37
GXG_00332,I,101901,C,A,90,2022,0.045000,756|2204,2033|4371,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,['YAL024C'],50
GXG_00339,I,102261,C,T,78,2020,0.039000,1116|2564,2393|4011,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,['YAL024C'],76
GXG_00346,I,102699,G,A,310,2016,0.154000,1554|3002,2831|3573,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,['YAL024C'],41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GXG_50437,XVI,899389,G,A,6,2018,0.002973,3625|805|1366|2655,2385,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,['YPR181C'],72
GXG_50440,XVI,899419,A,G,196,2014,0.097000,3655|775|1336|2625,2415,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,['YPR181C'],66
GXG_50446,XVI,899521,G,C,3,2018,0.001487,3757|673|1234|2523,2517,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,['YPR181C'],43


pool_num
35    115
36    115
37    115
38    115
39    115
40    115
41    115
42    115
43    115
44    115
45    115
46    116
47    116
48    116
49    116
50    116
51    116
52    116
53    116
54    116
55    116
56    116
57    116
58    116
59    116
60    116
61    116
62    116
63    116
64    116
65    116
66    116
67    116
68    114
69    114
70    114
71    114
72    114
73    114
74    114
75    114
76    114
77    114
Name: num_of_oligos, dtype: int64

Total number of oligos: 4957
Total number of variants: 2000


## GXE set
Oligos will be grouped according to condition (see assoc_traits_str)
- GXE: 94 pools

In [6]:
gxe_vars_file = "/home/users/rang/scratch/yeast/GxE/gxe_variants_final.txt"
gxe_vars = pd.read_csv(gxe_vars_file, sep='\t', index_col=0)
gxe_vars['num_of_oligos'] = gxe_vars['num_of_oligos'].astype(int)

# assign gxe variants to pools
gxe_vars = sort_variants_to_pools(vars_df = gxe_vars, 
                                  assignment_order = ['Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2',
                                                      'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Fructose;;1',
                                                      'Caffeine;15mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2',
                                                      'Cobalt_Chloride;2mM;2_Fructose;;1_Neomycin;5mg/mL;2',
                                                      'Lithium_Chloride;100mM;2_Neomycin;5mg/mL;2',
                                                      'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2',
                                                      'Caffeine;15mM;2_Cobalt_Chloride;2mM;2',
                                                      'Caffeine;15mM;2_Neomycin;5mg/mL;2',
                                                      'Cobalt_Chloride;2mM;2_Fructose;;1',
                                                      'Fluconazole;100uM;2_Fructose;;1',
                                                      'Caffeine;15mM;2_Fructose;;1',
                                                      'Lithium_Chloride;100mM;2',
                                                      'Cobalt_Chloride;2mM;2',
                                                      'Fluconazole;100uM;2',
                                                      'Neomycin;5mg/mL;2',
                                                      'Fructose;;1',
                                                      'Caffeine;15mM;2'],
                                  
                                  grouping_col = 'assoc_traits_str',
                                  
                                  num_of_pools_per_group = {'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2':3,
                                                            'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Fructose;;1':0,
                                                            'Caffeine;15mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2':3,
                                                            'Cobalt_Chloride;2mM;2_Fructose;;1_Neomycin;5mg/mL;2':1,
                                                            'Lithium_Chloride;100mM;2_Neomycin;5mg/mL;2':1,
                                                            'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2':1,
                                                            'Caffeine;15mM;2_Cobalt_Chloride;2mM;2':1,
                                                            'Caffeine;15mM;2_Neomycin;5mg/mL;2':0,
                                                            'Cobalt_Chloride;2mM;2_Fructose;;1':1,
                                                            'Fluconazole;100uM;2_Fructose;;1':0,
                                                            'Caffeine;15mM;2_Fructose;;1':0,
                                                            'Lithium_Chloride;100mM;2':14,
                                                            'Cobalt_Chloride;2mM;2':14,
                                                            'Fluconazole;100uM;2':13,
                                                            'Neomycin;5mg/mL;2':10,
                                                            'Fructose;;1':13,
                                                            'Caffeine;15mM;2':19},
                                                            
                                  spillover_assignment = {'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2':'Neomycin;5mg/mL;2',
                                                          'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Fructose;;1':'Cobalt_Chloride;2mM;2',
                                                          'Caffeine;15mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2':'Neomycin;5mg/mL;2',
                                                          'Cobalt_Chloride;2mM;2_Fructose;;1_Neomycin;5mg/mL;2':'Cobalt_Chloride;2mM;2',
                                                          'Lithium_Chloride;100mM;2_Neomycin;5mg/mL;2':'Lithium_Chloride;100mM;2',
                                                          'Cobalt_Chloride;2mM;2_Fluconazole;100uM;2':'Fluconazole;100uM;2',
                                                          'Caffeine;15mM;2_Neomycin;5mg/mL;2':'Neomycin;5mg/mL;2',
                                                          'Cobalt_Chloride;2mM;2_Fructose;;1':'Cobalt_Chloride;2mM;2',
                                                          'Fluconazole;100uM;2_Fructose;;1':'Fluconazole;100uM;2',
                                                          'Caffeine;15mM;2_Fructose;;1':'Caffeine;15mM;2',
                                                          'Lithium_Chloride;100mM;2':'Caffeine;15mM;2',
                                                          'Fructose;;1':'Caffeine;15mM;2'},
                                  max_pool_size = 118, first_pool = 84)

display(gxe_vars)
# check number of oligos in each pool
display(gxe_vars.groupby('pool_num').num_of_oligos.sum())
print("Total number of oligos:", gxe_vars.num_of_oligos.sum())
print("Total number of variants:", len(gxe_vars))

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,ACS,ADE,ADF,ADG,ADI,ADR,SACE_MAA,assoc_traits,assoc_traits_str,pool_num
var_id,,,,,,,,,,,,,,,,,,,,,
GXE_00023,I,192419,C,A,6,2020,0.002970,2|200,4687|3608|163,bidirectional_promoter,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,"['Cobalt_Chloride;2mM;2', 'Fluconazole;100uM;2...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...,85
GXE_00024,I,192424,GTTTGGATTACCTCT,ATTTGGATTACCTCT,39,2014,0.019000,7|181,4692|3613|168,bidirectional_promoter,...,0/0,NaN,0/0,0/0,0/0,0/0,0/0,"['Cobalt_Chloride;2mM;2', 'Fluconazole;100uM;2...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...,84
GXE_00026,I,192427,T,C,902,2000,0.451000,10|192,4695|3616|171,bidirectional_promoter,...,0/0,0/0,0/0,0/0,1/1,1/1,0/0,"['Cobalt_Chloride;2mM;2', 'Fluconazole;100uM;2...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...,85
GXE_00027,I,192429,G,C,1506,1998,0.754000,12|190,4697|3618|173,bidirectional_promoter,...,1/1,NaN,1/1,1/1,1/1,1/1,0/0,"['Cobalt_Chloride;2mM;2', 'Fluconazole;100uM;2...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...,86
GXE_00033,I,192466,AAAGGG,A,3,2020,0.001485,50|148,4735|3656|211,bidirectional_promoter,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,"['Cobalt_Chloride;2mM;2', 'Fluconazole;100uM;2...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GXE_17166,XVI,375077,T,C,3,2022,0.001484,425|4414,4043|2214|2156,missense_variant,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,"['Cobalt_Chloride;2mM;2', 'Fructose;;1', 'Neom...",Cobalt_Chloride;2mM;2_Fructose;;1_Neomycin;5mg...,109
GXE_17167,XVI,375078,C,A,8,2020,0.003960,424|4415,4042|2213|2157,stop_gained,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,"['Cobalt_Chloride;2mM;2', 'Fructose;;1', 'Neom...",Cobalt_Chloride;2mM;2_Fructose;;1_Neomycin;5mg...,109
GXE_17168,XVI,375080,G,A,31,2022,0.015000,422|4417,4040|2211|2159,missense_variant,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,"['Cobalt_Chloride;2mM;2', 'Fructose;;1', 'Neom...",Cobalt_Chloride;2mM;2_Fructose;;1_Neomycin;5mg...,90


pool_num
84     118
85     118
86     117
87     118
88     118
      ... 
173    117
174    117
175    117
176    118
177    118
Name: num_of_oligos, Length: 94, dtype: int64

Total number of oligos: 10983
Total number of variants: 4384
